### 1. Global Settings

In [1]:
import sys
sys.path.append("../../")
import json
import os
import surprise
import papermill as pm
import pandas as pd
import time
import yaml
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import rmse, precision_at_k, ndcg_at_k
from reco_utils.recommender.surprise.surprise_utils import compute_rating_predictions, compute_ranking_predictions

print("System version: {}".format(sys.version))
print("Surprise version: {}".format(surprise.__version__))

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
Surprise version: 1.0.6


### 2. Prepare Dataset
1. Download data and split into training, validation and test sets
2. Store the data sets to a local directory.

In [2]:
# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

In [3]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating"]
)

data.head()

URLError: <urlopen error [Errno 110] Connection timed out>

In [4]:
train, validation, test = python_random_split(data, [0.7, 0.15, 0.15])

In [5]:
DATA_DIR = 'aml_data'
os.makedirs(DATA_DIR, exist_ok=True)

TRAIN_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_train.pkl"
train.to_pickle(os.path.join(DATA_DIR, TRAIN_FILE_NAME))

VAL_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_val.pkl"
validation.to_pickle(os.path.join(DATA_DIR, VAL_FILE_NAME))

TEST_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_test.pkl"
test.to_pickle(os.path.join(DATA_DIR, TEST_FILE_NAME))

### 3. Prepare Hyperparameter Tuning 

We now prepare a training script [svd_training_nni.py](../../reco_utils/nni/svd_training.py) for the hyperparameter tuning, which will log our target metrics such as precision, NDCG, RMSE.
We define the arguments of the script and the search space for the hyperparameters. All the parameter values will be passed to our training script.

In [6]:
EXP_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_svd_model"
PRIMARY_METRIC = 'precision_at_k'
RATING_METRICS = ['rmse']
RANKING_METRICS = ['precision_at_k', 'ndcg_at_k']  
USERCOL = 'userID'
ITEMCOL = 'itemID'
RECOMMEND_SEEN = False
K = 10
RANDOM_STATE = 0
VERBOSE = True
NUM_EPOCHS = 30
BIASED = True

script_params = " ".join([
    '--datastore', os.path.join(os.getcwd(), 'aml_data'),
    '--train-datapath', TRAIN_FILE_NAME,
    '--validation-datapath', VAL_FILE_NAME,
    '--output-dir', './outputs',
    '--surprise-reader', 'ml-100k',
    '--rating-metrics', " ".join(RATING_METRICS),
    '--ranking-metrics', " ".join(RANKING_METRICS),
    '--usercol', USERCOL,
    '--itemcol', ITEMCOL,
    '--k', str(K),
    '--random-state', str(RANDOM_STATE),
    '--epochs', str(NUM_EPOCHS),
    '--primary-metric', PRIMARY_METRIC
])

if BIASED:
    script_params += ' --biased'
if VERBOSE:
    script_params += ' --verbose'
if RECOMMEND_SEEN:
    script_params += ' --recommend-seen'

In [7]:
# hyperparameters search space
# We do not set 'lr_all' and 'reg_all' because they will be overriden by the other lr_ and reg_ parameters

hyper_params = {
    'n_factors': {"_type": "choice", "_value": [10, 50, 100, 150, 200]},
    'init_mean': {"_type": "uniform", "_value": [-0.5, 0.5]},
    'init_std_dev': {"_type": "uniform", "_value": [0.01, 0.2]},
    'lr_bu': {"_type": "uniform", "_value": [1e-6, 0.1]}, 
    'lr_bi': {"_type": "uniform", "_value": [1e-6, 0.1]}, 
    'lr_pu': {"_type": "uniform", "_value": [1e-6, 0.1]}, 
    'lr_qi': {"_type": "uniform", "_value": [1e-6, 0.1]}, 
    'reg_bu': {"_type": "uniform", "_value": [1e-6, 1]},
    'reg_bi': {"_type": "uniform", "_value": [1e-6, 1]}, 
    'reg_pu': {"_type": "uniform", "_value": [1e-6, 1]}, 
    'reg_qi': {"_type": "uniform", "_value": [1e-6, 1]}
}

In [8]:
with open('../../reco_utils/nni/search_space_svd.json', 'w') as fp:
    json.dump(hyper_params, fp)

We also create a yaml file for the configuration of the trials and the tuning algorithm to be used. The tuning trials will be executed locally on a [Standard_D16_v3 virtual machine](https://docs.microsoft.com/en-us/azure/virtual-machines/windows/sizes-general#dv3-series-1) (16 vcpus, 64 GB memory).

In [17]:
config = {
    'authorName': 'default',
    'experimentName': 'surprise_svd',
    'trialConcurrency': 8,
    'maxExecDuration': '1h',
    'maxTrialNum': 100,
    'trainingServicePlatform': 'local',
    # The path to Search Space
    'searchSpacePath': 'search_space_svd.json',
    'useAnnotation': False,
    'tuner': {
        'builtinTunerName': 'TPE',
        'classArgs': {
            #choice: maximize, minimize
            'optimize_mode': 'maximize'
        }
    },
    # The path and the running command of trial
    'trial':  {
      'command': 'python3 svd_training.py' + " " + script_params,
      'codeDir': '.',
      'gpuNum': 0
    }
}
 
with open('../../reco_utils/nni/config_svd.yml', 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

### 5. Execute NNI Trials

cd ../../reco_utils/nni <br>
nnictl create --config config_svd.yml

You can see the experiment progress from this notebook by using the URL link.

### 6. Show Results

In [14]:
# Get best run and printout metrics
best_run = run.get_best_run_by_primary_metric()

best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

In [15]:
best_run_metrics

{'Number of epochs': 30,
 'rmse': 1.0343498081373697,
 'precision_at_k': 0.10000000000000002,
 'ndcg_at_k': 0.11498322243961594}

In [16]:
print(" ".join(parameter_values))

--datastore $AZUREML_DATAREFERENCE_workspaceblobstore --train-datapath data/movielens_100k_train.pkl --validation-datapath data/movielens_100k_val.pkl --output_dir ./outputs --surprise-reader ml-100k --rating-metrics rmse --ranking-metrics precision_at_k ndcg_at_k --usercol userID --itemcol itemID --k 10 --random-state 0 --epochs 30 --biased --verbose --n_factors 150 --init_mean -0.4163305768968 --init_std_dev 0.159711436379793 --lr_bu 0.0386753983834255 --lr_bi 4.48660045721016E-05 --lr_pu 0.0119378772073106 --lr_qi 0.0936873305814469 --reg_bu 0.385400397115581 --reg_bi 0.975251474623207 --reg_pu 0.906537637834819 --reg_qi 0.801240951271603


Now evaluate the metrics on the test data. To do this, get the SVD model that was saved as model.dump in the training script.

In [17]:
os.makedirs('aml_model', exist_ok=True)
best_run.download_file('outputs/model.dump', output_file_path='aml_model/')

In [18]:
svd = surprise.dump.load('aml_model/model.dump')[1]

In [19]:
test_results = {}
predictions = compute_rating_predictions(svd, test, usercol="userID", itemcol="itemID")
for metric in RATING_METRICS:
    test_results[metric] = eval(metric)(test, predictions)

all_predictions = compute_ranking_predictions(svd, train, usercol="userID", itemcol="itemID", recommend_seen=RECOMMEND_SEEN)
for metric in RANKING_METRICS:
    test_results[metric] = eval(metric)(test, all_predictions, col_prediction='prediction', k=K)

print(test_results)

{'rmse': 1.0331492610799313, 'precision_at_k': 0.09968017057569298, 'ndcg_at_k': 0.1160964958978592}


In [20]:
try:
    shutil.rmtree(SCRIPT_DIR)
    shutil.rmtree(DATA_DIR)
except (PermissionError, FileNotFoundError):
    pass

### 7. Concluding Remarks

We showed how to tune **all** the hyperparameters accepted by Surprise SVD simultaneously, by utilizing the Azure Machine Learning service. 
For example, training and evaluation of a single SVD model takes about 50 seconds on the 100k MovieLens data on a Standard D2_V2 VM. Searching through 100 different combinations of hyperparameters sequentially would take about 80 minutes whereas this notebook took less than half that. With AzureML, one can easily specify the size of the cluster according to the problem at hand and use Bayesian sampling to navigate efficiently through a large space of hyperparameters.

### References

* [Matrix factorization algorithms in Surprise](https://surprise.readthedocs.io/en/stable/matrix_factorization.html) 
* [Surprise SVD deep-dive notebook](../02_model/surprise_svd_deep_dive.ipynb)
* [Fine-tune natural language processing models using Azure Machine Learning service](https://azure.microsoft.com/en-us/blog/fine-tune-natural-language-processing-models-using-azure-machine-learning-service/)
* [Training, hyperparameter tune, and deploy with TensorFlow](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training-with-deep-learning/train-hyperparameter-tune-deploy-with-tensorflow/train-hyperparameter-tune-deploy-with-tensorflow.ipynb)
